# Pooler AI Training
Complete training setup with all required files

In [ ]:
# Install required packages
!pip install torch transformers google-cloud-storage tqdm

In [ ]:
# Create training script
%%writefile train_kaggle.py
import os
import torch
import logging
import json
from pathlib import Path
from datetime import datetime
from google.cloud import storage
from model.llm import PX1LLM
from local_trainer import LocalTrainer

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def setup_gcs_credentials():
    """Setup Google Cloud Storage credentials"""
    storage_client = storage.Client.create_anonymous_client()
    return storage_client

def download_dataset_from_gcs():
    """Download dataset from Google Cloud Storage"""
    logger.info("Downloading dataset from GCS")
    
    # Initialize GCS client
    storage_client = setup_gcs_credentials()
    bucket = storage_client.bucket("px1-dataset-v1")
    
    # Create dataset directory
    dataset_dir = "/kaggle/working/datasets"
    os.makedirs(dataset_dir, exist_ok=True)
    
    # Download all files from the training directory
    prefix = "training/dataset_v1/"
    blobs = bucket.list_blobs(prefix=prefix)
    
    for blob in blobs:
        # Create local directory structure
        local_path = os.path.join(dataset_dir, blob.name.replace(prefix, ""))
        os.makedirs(os.path.dirname(local_path), exist_ok=True)
        
        # Download file
        blob.download_to_filename(local_path)
        logger.info(f"Downloaded {blob.name} to {local_path}")
    
    return dataset_dir

def setup_kaggle_environment():
    """Setup Kaggle-specific configurations"""
    # Check if running on Kaggle
    if os.path.exists("/kaggle/working"):
        logger.info("Running on Kaggle environment")
        # Create necessary directories
        output_dirs = [
            "/kaggle/working/models",
            "/kaggle/working/outputs",
            "/kaggle/working/logs",
            "/kaggle/working/checkpoints"
        ]
        for dir_path in output_dirs:
            os.makedirs(dir_path, exist_ok=True)
            
        # Download dataset
        dataset_dir = download_dataset_from_gcs()
        logger.info(f"Dataset downloaded to {dataset_dir}")
        
        return "/kaggle/working"
    return "."

def save_outputs(work_dir: str, job_id: str, metrics: dict):
    """Save training outputs and metrics"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Save metrics
    metrics_file = Path(work_dir) / "outputs" / f"metrics_{timestamp}.json"
    with open(metrics_file, "w") as f:
        json.dump(metrics, f, indent=2)
    
    # Save model checkpoint
    checkpoint_file = Path(work_dir) / "checkpoints" / f"model_{timestamp}.pt"
    torch.save({
        'job_id': job_id,
        'metrics': metrics,
        'timestamp': timestamp
    }, checkpoint_file)

def main():
    # Setup environment
    work_dir = setup_kaggle_environment()
    
    # Initialize model and trainer
    model = PX1LLM()
    trainer = LocalTrainer(model)
    
    # Train the model
    metrics = trainer.train()
    
    # Save outputs
    save_outputs(work_dir, "kaggle_training", metrics)

if __name__ == "__main__":
    main()

In [ ]:
# Create model directory and model file
!mkdir -p model
%%writefile model/llm.py
import torch
import torch.nn as nn

class PX1LLM(nn.Module):
    def __init__(self):
        super().__init__()
        # Model architecture here
        self.model = nn.Sequential(
            nn.Linear(768, 512),
            nn.ReLU(),
            nn.Linear(512, 256)
        )
    
    def forward(self, x):
        return self.model(x)

In [ ]:
# Create trainer file
%%writefile local_trainer.py
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

class LocalTrainer:
    def __init__(self, model):
        self.model = model
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)
        self.optimizer = optim.Adam(model.parameters())
        self.criterion = nn.MSELoss()
    
    def train(self):
        print(f"Training on {self.device}")
        
        # Simple training loop for testing
        for epoch in range(5):
            # Generate dummy data
            x = torch.randn(32, 768).to(self.device)
            y = torch.randn(32, 256).to(self.device)
            
            # Forward pass
            outputs = self.model(x)
            loss = self.criterion(outputs, y)
            
            # Backward pass
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
            print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")
        
        return {"final_loss": loss.item()}

In [ ]:
# Check GPU setup
import torch
print("GPU Information:")
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Device:", torch.cuda.get_device_name())
    print("Number of GPUs:", torch.cuda.device_count())
    print("\nGPU Memory Info:")
    print(f"Total Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"Memory Allocated: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")
    print(f"Memory Cached: {torch.cuda.memory_reserved(0) / 1e9:.2f} GB")

In [ ]:
# Run training
!python train_kaggle.py